# Lab 3.1.4: 8B Model LoRA Fine-Tuning

**Module:** 3.1 - Large Language Model Fine-Tuning  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐☆☆

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Fine-tune a real 8B parameter LLM on your DGX Spark
- [ ] Combine LoRA/DoRA with NEFTune for optimal quality
- [ ] Use QLoRA (4-bit quantization) for efficient training
- [ ] Evaluate your fine-tuned model

---

## Prerequisites

- Completed: Labs 3.1.1-3.1.3 (LoRA Theory, DoRA, NEFTune)
- Model access: Request access to Llama 3.1 8B on HuggingFace (or use Mistral 7B)

---

## Real-World Context

### Your First Production Fine-Tune!

This is where everything comes together. You're going to:
1. Load a real 8B parameter model with 4-bit quantization
2. Apply DoRA adapters to attention layers
3. Enable NEFTune for better generalization
4. Fine-tune on a real instruction dataset
5. Evaluate and save your custom model

**On DGX Spark with 128GB unified memory:**
- 8B model in 4-bit: ~5-6GB
- LoRA adapters: ~100MB
- Training state: ~2-3GB
- **Plenty of room for larger batch sizes!**

---

## ELI5: What Are We Building?

> **Imagine you hired a brilliant new assistant (Llama 8B) who knows everything but needs to learn YOUR way of doing things.**
>
> Instead of sending them back to school for 4 years (full fine-tuning), you:
> 1. Give them a small notebook with YOUR preferences (LoRA/DoRA)
> 2. Have them practice with slight variations to build intuition (NEFTune)
> 3. Let them use a compressed version of their brain to save space (4-bit quantization)
>
> After a few hours of practice, they'll respond exactly how YOU want, while keeping all their original knowledge!

---

## Part 1: Environment Setup

First, let's make sure all dependencies are installed.

In [ ]:
# Check if running in NGC container (recommended)
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout[:500] if result.returncode == 0 else "GPU not detected")

In [ ]:
# Install required packages (if not in NGC container)
# In NGC container, most are pre-installed
# !pip install -q transformers>=4.40.0 peft>=0.10.0 trl>=0.8.0 datasets bitsandbytes accelerate

In [ ]:
import torch
import gc
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Total GPU Memory: {total_memory:.1f} GB")
    
    # DGX Spark detection
    if total_memory > 100:
        print("\n🚀 DGX Spark detected! You have 128GB unified memory.")
        print("   This means PLENTY of headroom for 8B models!")

# Clear any existing GPU memory
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# Import libraries
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
import os

print("All imports successful!")

---

## Part 2: Model Configuration

### Choose Your Model

We support two options:

| Model | Size | Access | Notes |
|-------|------|--------|-------|
| `meta-llama/Llama-3.1-8B-Instruct` | 8B | Requires approval | Best quality |
| `mistralai/Mistral-7B-Instruct-v0.2` | 7B | Open | Great alternative |
| `TinyLlama/TinyLlama-1.1B-Chat-v1.0` | 1.1B | Open | For quick testing |

In [ ]:
# ╔════════════════════════════════════════════════════════════════╗
# ║                    CONFIGURATION                                ║
# ╚════════════════════════════════════════════════════════════════╝

# Choose your model (uncomment one)
# MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"  # Requires approval
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # For quick testing
# MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"  # Open alternative

# Training configuration
OUTPUT_DIR = "./finetuned-model"
MAX_SEQ_LENGTH = 1024
BATCH_SIZE = 4  # DGX Spark can handle more!
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 2e-4
NUM_EPOCHS = 1  # Increase for production
WARMUP_RATIO = 0.1

# LoRA/DoRA configuration
LORA_RANK = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
USE_DORA = True  # Enable DoRA for +3.7 points!

# NEFTune configuration
NEFTUNE_ALPHA = 5.0  # 5-15 recommended

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  LoRA rank: {LORA_RANK} (alpha={LORA_ALPHA})")
print(f"  DoRA: {'Enabled' if USE_DORA else 'Disabled'}")
print(f"  NEFTune: α={NEFTUNE_ALPHA}")
print(f"  Batch size: {BATCH_SIZE} × {GRADIENT_ACCUMULATION} = {BATCH_SIZE * GRADIENT_ACCUMULATION} effective")

---

## Part 3: Load Model with 4-bit Quantization

We'll use QLoRA's NF4 quantization to reduce memory usage by 4x while maintaining quality.

In [ ]:
def get_memory_usage():
    """Get current GPU memory usage."""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        return f"Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB"
    return "GPU not available"

print(f"Before loading: {get_memory_usage()}")

In [ ]:
# QLoRA Configuration: 4-bit quantization with NF4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                     # Enable 4-bit loading
    bnb_4bit_quant_type="nf4",             # NormalFloat4 - best for LLMs
    bnb_4bit_compute_dtype=torch.bfloat16, # Compute in bfloat16 (DGX Spark native)
    bnb_4bit_use_double_quant=True,        # Quantize the quantization constants too
)

print("4-bit quantization config created")
print(f"  Quant type: NF4 (NormalFloat 4-bit)")
print(f"  Compute dtype: bfloat16")
print(f"  Double quantization: Enabled")

In [ ]:
# Load tokenizer
print(f"Loading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Set padding token if not present (common with Llama models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Tokenizer loaded:")
print(f"  Vocab size: {tokenizer.vocab_size:,}")
print(f"  Pad token: {tokenizer.pad_token}")

In [ ]:
# Load model with quantization
print(f"\nLoading model {MODEL_NAME} with 4-bit quantization...")
print("This may take a few minutes for larger models...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="sdpa",  # Use PyTorch's SDPA for efficiency
)

print(f"\nModel loaded!")
print(f"  {get_memory_usage()}")

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=True,  # Trade compute for memory
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel prepared for training:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable (before LoRA): {trainable_params:,}")
print(f"  Gradient checkpointing: Enabled")

---

## Part 4: Add LoRA/DoRA Adapters

Now we'll add our efficient adapters to the attention layers.

In [ ]:
# Find target modules (model-specific)
# Print some layer names to identify the pattern
print("Looking for attention layers...")
for name, _ in model.named_modules():
    if 'proj' in name.lower() or 'attention' in name.lower():
        print(f"  {name}")
        if len([n for n, _ in model.named_modules() if 'proj' in n.lower()]) > 10:
            print("  ...")
            break

In [ ]:
# LoRA/DoRA Configuration
lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention
        "gate_proj", "up_proj", "down_proj",      # MLP (optional but helps)
    ],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=USE_DORA,  # Enable DoRA!
)

print(f"{'DoRA' if USE_DORA else 'LoRA'} Configuration:")
print(f"  Rank: {LORA_RANK}")
print(f"  Alpha: {LORA_ALPHA} (scaling = {LORA_ALPHA/LORA_RANK})")
print(f"  Dropout: {LORA_DROPOUT}")
print(f"  Target modules: {lora_config.target_modules}")

In [ ]:
# Apply LoRA/DoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

print(f"\n{get_memory_usage()}")

### What Just Happened?

We've added small adapter matrices to each attention and MLP projection layer. The model now has:
- **Frozen base weights** (~1.1B for TinyLlama, ~8B for Llama)
- **Trainable LoRA/DoRA adapters** (~0.5-1% of total)

---

## Part 5: Prepare Dataset

We'll use a sample instruction-following dataset. For production, use your own data!

In [ ]:
# Load a sample instruction dataset
# Using a small subset for demonstration
print("Loading dataset...")

# Using Alpaca-style dataset
dataset = load_dataset("tatsu-lab/alpaca", split="train[:1000]")

print(f"Dataset loaded: {len(dataset)} examples")
print(f"\nExample:")
print(f"  Instruction: {dataset[0]['instruction'][:100]}...")
print(f"  Output: {dataset[0]['output'][:100]}...")

In [ ]:
# Format data for training
def format_instruction(example):
    """
    Format Alpaca-style data into a prompt.
    
    For Llama 3.1, we use the official chat template.
    For other models, we use a simple instruction format.
    """
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']
    
    if 'llama' in MODEL_NAME.lower() and '3' in MODEL_NAME:
        # Llama 3 format
        if input_text:
            prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{instruction}

Input: {input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output}<|eot_id|>"""
        else:
            prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output}<|eot_id|>"""
    else:
        # Generic instruction format
        if input_text:
            prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
        else:
            prompt = f"""### Instruction:
{instruction}

### Response:
{output}"""
    
    return {"text": prompt}


# Apply formatting
formatted_dataset = dataset.map(format_instruction)

print("Formatted example:")
print("="*50)
print(formatted_dataset[0]['text'][:500])
print("...")

---

## Part 6: Training with NEFTune

Now we'll set up training with all our optimizations!

In [ ]:
# Training configuration
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    
    # Batch size
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    
    # Training schedule
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,
    
    # Optimization
    optim="paged_adamw_8bit",  # Memory-efficient optimizer
    weight_decay=0.01,
    max_grad_norm=1.0,
    
    # Precision
    bf16=True,  # Use bfloat16 (DGX Spark native)
    
    # Sequence length
    max_seq_length=MAX_SEQ_LENGTH,
    packing=True,  # Pack multiple examples into one sequence
    
    # NEFTune! The magic ingredient
    neftune_noise_alpha=NEFTUNE_ALPHA,
    
    # Logging
    logging_steps=10,
    logging_first_step=True,
    
    # Saving
    save_strategy="epoch",
    save_total_limit=2,
    
    # Other
    report_to="none",  # Disable W&B for this demo
    seed=42,
)

print("Training configuration:")
print(f"  Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  NEFTune noise: α={NEFTUNE_ALPHA}")
print(f"  Precision: bfloat16")
print(f"  Max sequence length: {MAX_SEQ_LENGTH}")

In [ ]:
# Create trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
)

print("Trainer created!")
print(f"\n{get_memory_usage()}")

In [ ]:
# Train!
print("="*60)
print("Starting training...")
print(f"  Model: {MODEL_NAME}")
print(f"  Method: {'DoRA' if USE_DORA else 'LoRA'} + NEFTune")
print(f"  Dataset: {len(formatted_dataset)} examples")
print("="*60)

train_result = trainer.train()

print("\n" + "="*60)
print("Training complete!")
print("="*60)

In [ ]:
# Print training metrics
print("\nTraining Metrics:")
for key, value in train_result.metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

print(f"\n{get_memory_usage()}")

---

## Part 7: Test Your Fine-Tuned Model

Let's see how well your model learned!

In [ ]:
# Test generation
def generate_response(prompt: str, max_new_tokens: int = 256) -> str:
    """
    Generate a response from the fine-tuned model.
    """
    # Format prompt
    if 'llama' in MODEL_NAME.lower() and '3' in MODEL_NAME:
        formatted = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    else:
        formatted = f"""### Instruction:
{prompt}

### Response:
"""
    
    # Tokenize
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    
    # Generate
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the response part
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    elif "assistant" in response.lower():
        parts = response.split("assistant")
        response = parts[-1].strip() if len(parts) > 1 else response
    
    return response

In [ ]:
# Test with some prompts
test_prompts = [
    "Explain what machine learning is in simple terms.",
    "Write a short poem about artificial intelligence.",
    "What are three benefits of exercise?",
]

print("Testing fine-tuned model...")
print("="*60)

for prompt in test_prompts:
    print(f"\nPrompt: {prompt}")
    print("-"*40)
    response = generate_response(prompt)
    print(f"Response: {response}")
    print("="*60)

---

## Part 8: Save Your Model

Save the LoRA adapters for later use.

In [ ]:
# Save LoRA adapters
adapter_path = os.path.join(OUTPUT_DIR, "final_adapter")
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)

print(f"Adapter saved to: {adapter_path}")

# Show saved files
import os
print("\nSaved files:")
for f in os.listdir(adapter_path):
    size = os.path.getsize(os.path.join(adapter_path, f)) / 1e6
    print(f"  {f}: {size:.2f} MB")

In [ ]:
# How to load later:
load_example = f"""
# To load your fine-tuned model later:

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "{MODEL_NAME}",
    device_map="auto",
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("{adapter_path}")

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "{adapter_path}")

# Ready to use!
"""

print("How to load your model later:")
print(load_example)

---

## Part 9: Merge Weights (Optional)

For deployment, you can merge LoRA weights into the base model.

In [ ]:
# Optional: Merge LoRA weights into base model
# This removes the adapter overhead during inference

# Uncomment to merge:
# merged_model = model.merge_and_unload()
# merged_path = os.path.join(OUTPUT_DIR, "merged_model")
# merged_model.save_pretrained(merged_path)
# tokenizer.save_pretrained(merged_path)
# print(f"Merged model saved to: {merged_path}")

print("To merge weights, uncomment the code above.")
print("Merging creates a standalone model without adapter overhead.")

---

## Summary

Congratulations! You've successfully:

1. ✅ Loaded an 8B model with 4-bit quantization (QLoRA)
2. ✅ Added DoRA adapters (0.5-1% trainable params)
3. ✅ Enabled NEFTune for better generalization
4. ✅ Fine-tuned on instruction data
5. ✅ Tested your custom model
6. ✅ Saved adapters for later use

### Memory Usage on DGX Spark

| Component | Memory |
|-----------|--------|
| Base model (4-bit) | ~5-6GB |
| LoRA adapters | ~100MB |
| Training state | ~2-3GB |
| **Total** | **~8-10GB** |
| **Remaining** | **~118GB** |

**You have LOTS of headroom for larger batch sizes or bigger models!**

---

## Common Mistakes

### Mistake 1: Forgetting to Clear Cache Before Large Models

```python
# Wrong: Load new model without clearing
model = AutoModelForCausalLM.from_pretrained(...)  # OOM!

# Right: Clear cache first
torch.cuda.empty_cache()
gc.collect()
model = AutoModelForCausalLM.from_pretrained(...)
```

### Mistake 2: Wrong Padding Side

```python
# Wrong: Default padding (varies by model)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Right: Set padding side explicitly for causal LM
tokenizer.padding_side = "right"  # For training
# tokenizer.padding_side = "left"  # For generation
```

### Mistake 3: Not Using Gradient Checkpointing for Large Models

```python
# Wrong: No gradient checkpointing
model = get_peft_model(model, lora_config)  # May OOM on 70B

# Right: Enable gradient checkpointing
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
model = get_peft_model(model, lora_config)
```

---

## Cleanup

In [ ]:
# Clear GPU memory
del model, trainer
torch.cuda.empty_cache()
gc.collect()

print("Cleanup complete!")
print(f"{get_memory_usage()}")

---

## Next Steps

Ready for the big challenge? Continue to:

**[Lab 3.1.5: 70B Model QLoRA Fine-Tuning](lab-3.1.5-70b-qlora-finetuning.ipynb)** - The DGX Spark showcase! Fine-tune a 70B model on your desktop!